In [24]:
!pip install pysrt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [26]:
import os
os.chdir('/content/drive/MyDrive/nlp/Prof_Blas')
os.getcwd()

'/content/drive/MyDrive/nlp/Prof_Blas'

## Import Packages

In [27]:
import json
import re
import pysrt
import pandas as pd

In [28]:
configJsonPath = "config.json"
category = ["Profanity", "Blasphemy"]


category = [x.lower() for x in category]
with open(configJsonPath, 'r') as f:
    configJson = json.load(f)

In [29]:
def toLower(dictionary):

    def tryIterate(k):
        return lowerByLevel(k) if isinstance(k, dict) else k

    def tryLower(k):
        return k.lower() if isinstance(k, str) else k

    def lowerByLevel(data):
        return dict((tryLower(k), tryIterate(v)) for k, v in data.items())

    return lowerByLevel(dictionary)

In [19]:
def context_match(element, text):
    label = element['label']
    context = element['context'].replace('*','\*').replace('$','\$')
#     print(context)
    matchCaseInsensitive = re.compile(context, re.IGNORECASE)
    matched = matchCaseInsensitive.search(text)
#     matches = re.finditer(context, text, re.MULTILINE)
#     for match in matches:
#         print(match)
    if matched:
        return label

In [30]:
def parser(text):
    matched_labels = []
    matched_class = []
    for cat in range(0,len(configJson)):
        offensive_list = toLower(configJson[cat])
        off_class = list(offensive_list.keys())[0]
        if off_class in category:
            for idx in range(0,len(offensive_list[off_class])):
                element = toLower(offensive_list[off_class][idx])
                if context_match(element, text):
                    matched_labels.append(context_match(element, text))
                    matched_class.append(off_class.title())

    return matched_labels, matched_class
            

In [21]:
# parser(r"For God's Sake")

In [31]:
def main(srt_file_path, srt_file_name):
    df = pd.DataFrame()
    subs = pysrt.open(srt_file_path + srt_file_name)
    for sub in subs:
        word_list, pred_list = parser(sub.text)
        # print(sub.text)
        df = df.append({'text': sub.text, 'start':sub.start, 'end': sub.end, 'word': ','.join(word_list), 'pred': ','.join(pred_list)}, ignore_index=True )
        # print()
    df = df.reindex(['text', 'start', 'end', 'word', 'pred'], axis=1)
    return df

## Run

In [32]:
srt_file_path = r"srt_files/"
srt_file_name = r"Bully 720pED.srt"


output_df = main(srt_file_path, srt_file_name)
file_name = 'output/' + ''.join(srt_file_name.split('.')[:-1]) + ".csv"
output_df.to_csv(file_name,index = False)
output_df

text         start           end  \
0   That bastard!\nClaire wanted to scream.  00:00:55,797  00:00:58,277   
1                      Marty honey, dinner.  00:00:59,334  00:01:01,644   
2                   I'm sick\nof this crap.  00:01:08,443  00:01:10,514   
3                          <i>His legs?</i>  00:01:41,643  00:01:42,917   
4       <i>That seemed to piss her off.</i>  00:01:47,849  00:01:49,487   
..                                      ...           ...           ...   
95                           Fuck you, man.  00:11:18,920  00:11:20,126   
96          I'm sorry, man.\nLet's just go.  00:11:31,566  00:11:34,342   
97                     Let's go, I'm sorry.  00:11:35,536  00:11:37,379   
98   Get your fuckin' hands\noff me, Bobby.  00:11:37,438  00:11:39,509   
99                        I'm really sorry.  00:11:40,641  00:11:42,211   

       word       pred  
0   Bastard  Profanity  
1                       
2                       
3                       
4      Piss  Profanity  
..      ...        ...  
95     Fuck  Profanity  
96                      
97                      
98     Fuck  Profanity  
99                      

[100 rows x 5 columns]